In [19]:
import pandas as pd
import numpy as np



In [21]:
matches = pd.read_csv("matches (1).csv")   # use your original file name
deliveries = pd.read_csv("deliveries.csv")

# Data Cleaning in EDA

In [22]:
# Step 3: Initial Look
print(matches.shape)
print(deliveries.shape)


(636, 18)
(150460, 21)


In [24]:
matches.head()
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


## Check Missing Values

In [25]:
# Count missing values per column
print("Matches dataset missing values:")
print(matches.isnull().sum())


Matches dataset missing values:
id                   0
season               0
city                 7
date                 0
team1                0
team2                0
toss_winner          0
toss_decision        0
result               0
dl_applied           0
winner               3
win_by_runs          0
win_by_wickets       0
player_of_match      3
venue                0
umpire1              1
umpire2              1
umpire3            636
dtype: int64


In [26]:
print("\nDeliveries dataset missing values:")
print(deliveries.isnull().sum())


Deliveries dataset missing values:
match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
is_super_over            0
wide_runs                0
bye_runs                 0
legbye_runs              0
noball_runs              0
penalty_runs             0
batsman_runs             0
extra_runs               0
total_runs               0
player_dismissed    143022
dismissal_kind      143022
fielder             145091
dtype: int64


## Handle Duplicates

In [27]:
# Check duplicates
print("Duplicate rows in matches:", matches.duplicated().sum())
print("Duplicate rows in deliveries:", deliveries.duplicated().sum())


Duplicate rows in matches: 0
Duplicate rows in deliveries: 1


In [28]:
# Drop if necessary
matches.drop_duplicates(inplace=True)
deliveries.drop_duplicates(inplace=True)

## Clean String Columns (whitespace, casing)

In [29]:
# Strip whitespace and fix casing
for col in matches.select_dtypes(include='object'):
    matches[col] = matches[col].str.strip()

for col in deliveries.select_dtypes(include='object'):
    deliveries[col] = deliveries[col].str.strip()

## Convert Data Types

In [30]:
# Convert date column to datetime
matches['date'] = pd.to_datetime(matches['date'], errors='coerce')


In [31]:
# Ensure numeric columns are correct type
matches['season'] = pd.to_numeric(matches['season'], errors='coerce')
deliveries['over'] = pd.to_numeric(deliveries['over'], errors='coerce')


## Check Referential Integrity

In [32]:
# deliveries.match_id should exist in matches.id
invalid_ids = set(deliveries['match_id']) - set(matches['id'])
print("Deliveries with invalid match_id:", len(invalid_ids))

Deliveries with invalid match_id: 0


## Handle Nulls

In [35]:
# Safer fix for missing city values
matches['city'] = matches['city'].fillna(matches['venue'])

In [36]:
# Fill missing umpire names with 'Unknown'
matches[['umpire1','umpire2','umpire3']] = matches[['umpire1','umpire2','umpire3']].fillna('Unknown')

## Save Cleaned Data

In [39]:
matches.to_csv("matches_clean.csv", index=False)
deliveries.to_csv("deliveries_clean.csv", index=False)


# Highest and lowest scorers (batsman total runs)

In [40]:
batsman_runs = deliveries.groupby("batsman")["batsman_runs"].sum().sort_values(ascending=False)
highest_scorers = batsman_runs.head(10).reset_index()
lowest_scorers = batsman_runs[batsman_runs > 0].tail(10).reset_index()